In [1]:
from keras.models import *
from keras.layers import *
from keras.callbacks import *
from keras.optimizers import *
from keras.applications import *
from keras.preprocessing.image import *

from glob import glob
from tqdm import tqdm
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from sklearn.metrics import *

import multiprocessing
from multiprocessing.dummy import Pool

%matplotlib inline

# 图片展示函数
def show(image):
    img = mpimg.imread(image)
    plt.imshow(img)

Using TensorFlow backend.


In [2]:
width = 224

fnames = glob('data/*/*.jpg')
class0 = len(glob('data/normal/*.jpg'))
class1 = len(glob('data/abnormal/*.jpg'))

n = len(fnames)

X = np.zeros((n, width, width, 3), dtype=np.uint8)
y = np.zeros((n, 1), dtype=np.uint8)

def f(index):
    return index, cv2.resize(cv2.imread(fnames[index]), (width, width))

with Pool(multiprocessing.cpu_count()) as pool:
    with tqdm(pool.imap_unordered(f, range(n)), total=n) as pbar:
        for i, img in pbar:
            X[i] = img
            if 'abnormal' in fnames[i]:
                y[i] = 1
            else:
                y[i] = 0

100%|██████████| 2022/2022 [03:02<00:00, 13.69it/s]


In [3]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

In [4]:
class RocAucMetricCallback(Callback):
    def __init__(self):
        super(RocAucMetricCallback, self).__init__()

    def on_epoch_end(self, epoch, logs={}):
        logs['roc_auc_val']=float('-inf')
        y_pred = model.predict(X_valid)
        auc = roc_auc_score(y_valid, y_pred)
        acc = accuracy_score(y_valid[:,0], y_pred[:,0] > 0.5)
        print(f'ACC: {acc:.4f} AUC: {auc:.4f}')

In [5]:
input_tensor = Input((width, width, 3))

x = Lambda(resnet50.preprocess_input)(input_tensor)
base_model = ResNet50(include_top=False, weights='imagenet', input_tensor=x)

# for layers in base_model.layers:
#     layers.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)
model = Model(input_tensor, x)

In [ ]:
model.compile(optimizer=Adam(1e-3), 
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=8, epochs=7, callbacks=[RocAucMetricCallback()])

Epoch 1/7
1617/1617 [==============================] - 622s 385ms/step - loss: 0.7980 - acc: 0.6327
Epoch 2/7
1617/1617 [==============================] - 594s 367ms/step - loss: 0.6616 - acc: 0.6636
Epoch 3/7
1617/1617 [==============================] - 602s 372ms/step - loss: 0.6363 - acc: 0.6741
Epoch 4/7
1488/1617 [==========================>...] - ETA: 45s - loss: 0.6367 - acc: 0.6828

In [ ]:
y_pred = model.predict(X_valid, verbose=1)
roc_auc_score(y_valid, y_pred), accuracy_score(y_valid[:,0], y_pred[:,0] > 0.5)

In [8]:
confusion_matrix(y_valid > 0.5, y_pred > 0.5)

array([[177,  80],
       [ 69,  79]])

In [9]:
fnames = glob('xuelang_round1_test_a_20180709/*.jpg')
n = len(fnames)